In [1]:
from pathlib import Path
import argparse
import json
import os
import random
import signal
import sys
import time
import urllib

from torch import nn, optim
from torchvision import datasets, transforms
import torch

import resnet

import numpy as np
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
import json

import glob
from geopy.geocoders import Nominatim
import re
import pandas as pd
from torchvision.io import read_image
from sklearn.preprocessing import LabelEncoder
from torch.utils.tensorboard import SummaryWriter
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import math
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True   #OTHERWISE TRUNCATED IMAGE FILE ERROR SOMEWHERE IN ENUMERATE(DATALOADER)!!!!

import resnet

import torchvision

In [2]:
class Args():
    
    def __init__(self):
        
        self.data_dir = '../../ids_and_labels_train_repeated.csv'
        self.train_percent = 5
        self.pretrained = './exp/resnet18.pth'
        self.exp_dir = 'exp'
        self.print_freq = 100
        self.arch = 'resnet18'
        self.epochs = 20
        self.batch_size = 16
        self.lr_backbone = 0.03
        self.lr_head = 0.08
        self.weight_decay = 0.0
        self.weights = 'finetune'
        self.workers = 10
        self.rank = 0
        self.dist_url = f"tcp://localhost:{random.randrange(49152, 65535)}"
        self.world_size = 1

In [3]:
args = Args()

In [7]:
image_id_and_labels = pd.read_csv('../../ids_and_labels.csv',delimiter=',',header=1)
#print(image_id_and_labels.values)
image_id_and_labels_arr = np.array(image_id_and_labels.values)
#print(image_id_and_labels_arr)

label_encoder = LabelEncoder()
image_id_and_labels_encoded = image_id_and_labels_arr.copy()
label_encoder.fit(image_id_and_labels_encoded[:,3])
encoded_countries = label_encoder.transform(image_id_and_labels_encoded[:,3])
#print(encoded_countries)
image_id_and_labels_encoded[:,3] = encoded_countries
print(image_id_and_labels_encoded)

[['global_monthly_2018_09_mosaic_L15-0331E-1257N_1327_3160_13.tif' nan
  'California' 18]
 ['global_monthly_2018_06_mosaic_L15-0331E-1257N_1327_3160_13.tif' nan
  'California' 18]
 ['global_monthly_2018_11_mosaic_L15-0331E-1257N_1327_3160_13.tif' nan
  'California' 18]
 ...
 ['global_monthly_2019_01_mosaic_L15-1848E-0793N_7394_5018_13.tif'
  'Melbourne' 'Victoria' 1]
 ['global_monthly_2018_12_mosaic_L15-1848E-0793N_7394_5018_13.tif'
  'Melbourne' 'Victoria' 1]
 ['global_monthly_2019_06_mosaic_L15-1848E-0793N_7394_5018_13.tif'
  'Melbourne' 'Victoria' 1]]


In [8]:
stratSplit = StratifiedShuffleSplit(n_splits=1, test_size = args.train_percent*0.01, random_state=42)

for train_idx, test_idx in stratSplit.split(image_id_and_labels_encoded, np.zeros(len(image_id_and_labels_encoded))):
    ids_labels_train, ids_labels_train_subset = image_id_and_labels_encoded[train_idx], image_id_and_labels_encoded[test_idx]

print(len(ids_labels_train_subset))
np.savetxt("ids_and_labels_train_subset.csv", ids_labels_train_subset, delimiter=",", fmt='%s')

72


In [9]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [10]:
def main_worker(gpu, args):

    torch.cuda.set_device(gpu)
    torch.backends.cudnn.benchmark = True

    backbone, embedding = resnet.__dict__[args.arch](zero_init_residual=True)
    state_dict = torch.load(args.pretrained, map_location="cpu")
    if "model" in state_dict:
        state_dict = state_dict["model"]
        state_dict = {
            key.replace("module.backbone.", ""): value
            for (key, value) in state_dict.items()
        }
    backbone.load_state_dict(state_dict, strict=False)

    head = nn.Linear(embedding, 31)                            #CHANGE ACCORDING TO NUMBER OF CLASSES!!!!!
    head.weight.data.normal_(mean=0.0, std=0.01)
    head.bias.data.zero_()
    model = nn.Sequential(backbone, head)
    model.cuda(gpu)

    if args.weights == "freeze":
        backbone.requires_grad_(False)
        head.requires_grad_(True)

    criterion = nn.CrossEntropyLoss().cuda(gpu)

    param_groups = [dict(params=head.parameters(), lr=args.lr_head)]
    
    if args.weights == "finetune":
        param_groups.append(dict(params=backbone.parameters(), lr=args.lr_backbone))
        
    optimizer = optim.SGD(param_groups, 0, momentum=0.9, weight_decay=args.weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs)

    
    start_epoch = 0
    best_acc = argparse.Namespace(top1=0, top5=0)

    # Data loading code
    
    class SN7Dataset(Dataset):
    
        def __init__(self, transform=None, train=True):
        
            if train == True:
                self.img_ids_labels = pd.read_csv("ids_and_labels_train_subset.csv", header = None) 
            else:
                self.img_ids_labels = pd.read_csv("../../ids_and_labels_val.csv", header = None)
            
            self.img_dir = '/local/home/stuff/datasets/Challenge_7/train'
            self.transform = transform
        
        def __getitem__(self, idx):
        
            image_id = self.img_ids_labels.iloc[idx, 0]
            pattern = "mosaic_(.*?).tif"
            location_id = re.search(pattern, image_id).group(1)
            #print(image_id)
            #print(location_id)
            img_path = os.path.join(self.img_dir, location_id, 'images', image_id )
            image = torchvision.transforms.ToTensor()(Image.open(img_path))[0:3,:,:]  #TAKE RGB CHANNELS ONLY FOR RESNET COMPATIBILITY!!!
        
            image_padded = torch.nn.functional.pad(image, pad=(0, 1024 - image.shape[2], 0, 1024 - image.shape[1]))
            patches = image_padded.unfold(1, 256, 256).unfold(2, 256, 256)
            patches = patches.reshape(3, -1, 256, 256)
            patches = patches.permute(1,0,2,3)
        
            label = torch.from_numpy(np.asarray(self.img_ids_labels.iloc[idx, 3]))*torch.ones(patches.size(0))
        
            #patches = patches.view(-1, 3, 256, 256)
            #label = label.flatten()
            #label = label.type(torch.LongTensor)
        
            
            patches = self.transform(patches)
            
            return patches, label
        
        def __len__(self):
        
            return len(self.img_ids_labels)
    
    
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )

   
    train_transforms = transforms.Compose(
            [
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                #transforms.ToTensor(),
                normalize,
            ]
        )
    
    val_transforms = transforms.Compose(
            [
                transforms.Resize(256),
                transforms.CenterCrop(224),
                #transforms.ToTensor(),
                normalize,
            ]
        )

    #DATASSET AND LOADERS
    
    train_dataset = SN7Dataset(train=True, transform = train_transforms)
    val_dataset = SN7Dataset(train=False, transform = val_transforms)
    
    kwargs = dict(
        batch_size=args.batch_size // args.world_size,
        num_workers=args.workers,
        pin_memory=True, shuffle=True
    )
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, **kwargs
    )
    val_loader = torch.utils.data.DataLoader(val_dataset, **kwargs)

    start_time = time.time()
    
    #WEIGHT FINETUNING/TRAINIG
    
    running_loss = 0.0
    checkpoints = 30
    n_total_steps = len(train_loader)
    
    for epoch in range(start_epoch, args.epochs):
        # TRAIN
        
        if args.weights == "finetune":
            model.train()
        elif args.weights == "freeze":
            model.eval()
        else:
            assert False

        for step, (images, target) in enumerate(
            train_loader, start=epoch * len(train_loader)
        ):
            images = images.view(-1, 3, 224, 224)    
            target = target.flatten()
            target = target.type(torch.LongTensor)
            
            output = model(images.cuda(gpu, non_blocking=True))
            loss = criterion(output, target.cuda(gpu, non_blocking=True))
            
            running_loss += loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (step+1) % checkpoints == 0:
            
                print (f'Epoch [{epoch+1}/{args.epochs}], Step [{step+1}/{n_total_steps}], Loss: {loss.item():.4f}')
                print('training_loss', running_loss/checkpoints, epoch * n_total_steps + step)
                running_loss = 0.0

        # EVALUATE
        
        model.eval()
        if args.rank == 0:
            top1 = AverageMeter("Acc@1")
            top5 = AverageMeter("Acc@5")
            with torch.no_grad():
                for images, target in val_loader:
                    
                    images = images.view(-1, 3, 224, 224)    
                    target = target.flatten()
                    target = target.type(torch.LongTensor)
            
                    output = model(images.cuda(gpu, non_blocking=True))
                    acc1, acc5 = accuracy(
                        output, target.cuda(gpu, non_blocking=True), topk=(1, 5)
                    )
                    top1.update(acc1[0].item(), images.size(0))
                    top5.update(acc5[0].item(), images.size(0))
            best_acc.top1 = max(best_acc.top1, top1.avg)
            best_acc.top5 = max(best_acc.top5, top5.avg)
            print('best_acc.top1', best_acc.top1)
            print('best_acc.top5', best_acc.top5)
            
        scheduler.step()
        
    ids_labels_val = pd.read_csv('../../ids_and_labels_val.csv',delimiter=',',header=1)
    ids_labels_val = np.array(ids_labels_val)
    print(ids_labels_val.shape)
    val_classes = np.arange(len(np.unique(ids_labels_val[:,3].astype('int'))))
    classes = np.unique(val_classes)
    print(len(classes))

    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        n_class_correct = [0 for i in range(len(classes))]
        n_class_samples = [0 for i in range(len(classes))]
        for images, labels in val_loader:
        
            images = images.view(-1, 3, 224, 224)     
            labels = labels.flatten()
            labels = labels.type(torch.LongTensor)
        
            images = images.to(gpu)  
            #print(labels.shape)
            labels = labels.to(gpu)
            outputs = model(images)
            # max returns (value ,index)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()
        
            for i in range(len(labels)):
                label = labels[i]
                pred = predicted[i]
                if (label == pred):
                    n_class_correct[label] += 1
                n_class_samples[label] += 1

        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy of the network: {acc} %')

        for i in range(len(classes)):
            acc = 100.0 * n_class_correct[i] / n_class_samples[i]
            print(f'Accuracy of {classes[i]}: {acc} %')
        

def handle_sigusr1(signum, frame):
    os.system(f'scontrol requeue {os.getenv("SLURM_JOB_ID")}')
    exit()


def handle_sigterm(signum, frame):
    pass


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=":f"):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = "{name} {val" + self.fmt + "} ({avg" + self.fmt + "})"
        return fmtstr.format(**self.__dict__)
    

In [11]:
main_worker('cuda:2', args)

best_acc.top1 1.9662921348314606
best_acc.top5 9.55056179775281
best_acc.top1 1.9662921348314606
best_acc.top5 9.55056179775281
best_acc.top1 1.9662921348314606
best_acc.top5 9.55056179775281
best_acc.top1 1.9662921348314606
best_acc.top5 9.55056179775281
best_acc.top1 1.9662921348314606
best_acc.top5 9.55056179775281
Epoch [6/20], Step [30/5], Loss: 88.6476
training_loss 3976.351356681188 54
best_acc.top1 1.9662921348314606
best_acc.top5 9.55056179775281
best_acc.top1 1.9662921348314606
best_acc.top5 9.55056179775281
best_acc.top1 1.9662921348314606
best_acc.top5 9.55056179775281
best_acc.top1 1.9662921348314606
best_acc.top5 9.55056179775281
best_acc.top1 1.9662921348314606
best_acc.top5 9.55056179775281
best_acc.top1 1.9662921348314606
best_acc.top5 9.55056179775281
Epoch [12/20], Step [60/5], Loss: 63.2206
training_loss 137.72008034388224 114
best_acc.top1 1.9662921348314606
best_acc.top5 9.55056179775281
best_acc.top1 1.9662921348314606
best_acc.top5 9.55056179775281
best_acc.top1

In [62]:
ids_labels_val = pd.read_csv('../../ids_and_labels_val.csv',delimiter=',',header=1)
ids_labels_val = np.array(ids_labels_val)
print(ids_labels_val.shape)
val_classes = np.arange(len(np.unique(ids_labels_val[:,3].astype('int'))))
classes = np.unique(val_classes)
print(len(classes))

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(len(classes))]
    n_class_samples = [0 for i in range(len(classes))]
    for images, labels in val_loader:
        
        images = images.view(-1, 3, 224, 224)     
        labels = labels.flatten()
        labels = labels.type(torch.LongTensor)
        
        images = images.to(device)  
        #print(labels.shape)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(len(labels)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(len(classes)):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

(354, 4)
31


NameError: name 'val_loader' is not defined